#**TrDesign**
<font color='grey' size='1.5'> Adapted from [Dr. Sergey Ovchinnikov](https://github.com/sokrypton/ColabDesign)and modified by Parisa Hosseinzadeh for *Protein Engineering and Design*, Winter 2022

In this activity, we will be designing completely new proteins using network hallucination. For details see this [link](https://github.com/gjoni/trDesign) and read [this paper](https://www.nature.com/articles/s41586-021-04184-w).

In [1]:
#@title Download master code

%%bash
wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/beta/02-GD/models.py
wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/beta/02-GD/utils.py
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/design/6MRR.pdb
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/design/to_pdb.py
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/models.zip
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/bkgr_models.zip
unzip -qqo models.zip
unzip -qqo bkgr_models.zip
pip -q install py3Dmol

In [2]:
#@title Import necessary modules

import numpy as numpy
from scipy.special import softmax
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.gridspec import GridSpec
from IPython.display import HTML
import py3Dmol

from utils import *
from models import *
from to_pdb import *

In [3]:
#@title Define plot functions

def plot_feat(x):
  '''plot each feature'''
  plt.figure(figsize=(4*4,4))
  for n,(k,v) in enumerate(split_feat(x).items()):
    plt.subplot(1,4,n+1); plt.title(k)
    plt.imshow(np.squeeze(v).argmax(-1),cmap="binary")
  plt.show()

def animate(design, do_ic=False):
  '''animate given trajectory'''
  fig, imgs = plt.figure(figsize=(6,3), dpi=100),[]
  gs = GridSpec(1,4, figure=fig)
  if do_ic:
    ax1 = fig.add_subplot(gs[0,0:2])
    ax1.axis(False)
    ax1.title.set_text("distance matrix")
    ax2 = fig.add_subplot(gs[0,2:4])
    ax2.axis(False)
    ax2.title.set_text("coevolution matrix")
  else:
    ax1 = fig.add_subplot(gs[0,0:2])
    ax1.axis(False);ax1.title.set_text("distance matrix")
    ax2 = fig.add_subplot(gs[0,2])
    ax2.axis(False);ax2.title.set_text("sequence")
    ax3 = fig.add_subplot(gs[0,3])
    ax3.axis(False);ax3.title.set_text("gradient")

  # go through each step along traj
  for k,out in enumerate(design["traj"]):
    # plot distance (contact map)
    pred = split_feat(out["feat"])["dist"][0].argmax(-1)
    # plot pssm (probability of amino acid at each position)
    im1 = ax1.imshow(pred, animated=True,cmap="binary")
    if do_ic:
      ic = inv_cov(softmax(out["I"],-1)[0].argmax(-1))
      im2 = ax2.imshow(ic, animated=True,cmap="binary")
      imgs.append([im1,im2])
    else:
      pssm = softmax(out["I"],-1)[0,0]
      im2 = ax2.imshow(pssm, animated=True,vmin=0,vmax=1,cmap="binary")
      grad = out["grad"][0,0]
      im3 = ax3.imshow(grad, animated=True,vmin=-0.1,vmax=0.1,cmap="bwr")
      imgs.append([im1,im2,im3])
  ani = animation.ArtistAnimation(fig, imgs, blit=True, interval=40)
  plt.close()
  return ani.to_html5_video()

def get_pdb(design, pdb_filename="out.pdb", mds="classic"):
  '''given features, return approx. 3D structure'''
  if "I" in design: seq = design["I"]
  seq = N_to_AA(np.squeeze(seq).argmax(-1))[0]
  xyz, dm = feat_to_xyz(np.squeeze(design["feat"]), mds=mds)
  save_PDB(pdb_filename, xyz, dm, seq)

  p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  p.addModel(open(pdb_filename,'r').read(),'pdb')
  p.setStyle({'cartoon': {'color':'spectrum'}})
  p.zoomTo()
  p.show()

#**PART 1** - Structure prediction
For Part 1, we'll using TrRosetta NN to predict structure from single sequence (for structure prediction from MSA, see part 5)

In [ ]:
#@title Initialize the model
model = mk_design_model(serial=True)

In [5]:
#@title Input sequence

seq = "GWSTELEKHREELKEFLKKEGITNVEIRIDNGRLEVRVEGGTERLKRFLEELRQKLEKKGYTVDIKIE" #@param {type:"string"}



In [ ]:
#@title Running the prediction

_seq = np.eye(20)[AA_to_N(seq)]
#uncomment this if you want to see the one-hot encoding plot
#plt.imshow(_seq.T, cmap="binary")
# use model to make prediction
preds = model.predict({"I":_seq[None,None]})

In [ ]:
#@title Plot predicted maps

plot_feat(preds["feat"])

What are these "features"?
These are the 6D transformation for every pair of residues that describe a protein structure.
Using this representation of a protein, one could theorically reconstruct the 3D structure exactly.

![6D transformation](https://raw.githubusercontent.com/gjoni/trDesign/master/02-GD/notebooks/6D.png)

**But.. but, how do I see my protein... in 3D?**
Normally we'd input the predicted constraints (or features) into PyRosetta for structure prediction... but since this is quite expensive to run in Google Colab, for demo purposes let's use a quick approximation (secret recipe) that converts the pairwise 6D features into a full 3D structure!

In [ ]:
#@title Visualize the protein

get_pdb(preds,"foldit1.pdb")

### **EXERCISE**
Try change the sequence, how does this change the structure?

In [ ]:
seq = "ALKALKALKALKALKALKALKALK" #@param {type:"string"}

_seq = np.eye(20)[AA_to_N(seq)]
preds = model.predict({"I":_seq[None,None]})
plot_feat(preds["feat"])
get_pdb(preds,"random.pdb")

#**PART 2** - Fixed backbone design!
For Part 2, we'll using TrDesign NN to design a new sequence for a given protein backbone!

![the protocol](https://raw.githubusercontent.com/gjoni/trDesign/master/02-GD/notebooks/figure_1.png)

In [ ]:
#@title step 1: load features from protein structure (PDB)
#@markdown Give an input pdb as a backbone guide
#@markdown You can see the target maps below.

p_name = "6MRR.pdb" #@param {type:"string"}

pdb = prep_input(p_name, chain="A")
_feat = pdb["feat"]
_seq = np.eye(20)[AA_to_N(pdb["seq"])]

plot_feat(_feat)

step 2: initialize the design model

In [ ]:
#@title step 2: initialize the design model
#@markdown **WARNING**, for demo we use n_models=1, 
#@markdown in practice you should ALWAYS use n_models=5

model = mk_design_model(add_pdb=True, n_models=1)

In [ ]:
#@title step 3: design a new sequence 
#@markdown In other words, optimize sequence to match target features derived from the PDB file

design = model.design(inputs={"pdb":_feat[None]}, return_traj=True)

In [ ]:
#@title step 4: analyze
#@markdown Let's take a look at the loss first.
#@markdown The reported loss it the CCE (categorical cross entropy)
#@markdown between the predicted features and target features

#@markdown `loss = -target * log(pred)`

-0.25 * (_feat * np.log(design["feat"])).sum(-1).mean()

In [ ]:
#@markdown Let's take a look at designed maps.
plot_feat(design["feat"])

In [ ]:
#@markdown Let's check the designed sequence.
#@markdown You can predict its structure using AlphaFold2
#@markdown if you're curious.

N_to_AA(design["I"][0].argmax(-1))

In [ ]:
#@markdown Let's checkout a movie of how this works.
HTML(animate(design))

In [ ]:
#@title Visualization

get_pdb(design,"6MRR_redesign.pdb",mds="metric")

#**PART 3** - hallucination time!
For Part 3, we'll use TrDesign NN to "hallucinate" or design a new (or denovo) protein (both sequence and structure), that doesn't exist in nature.

In [ ]:
#@title step 1: compute a background distribution
#@markdown This describes what a protein structure looks like, for given length.

length = 50 # feel free to change to see what happens!
_bkg = get_bkg([length])[length]

plot_feat(_bkg)

_bkg_dist = split_feat(_bkg)["dist"]
# for k in range(1,6):
#   plt.plot(np.linspace(2,20,36),
#            _bkg_dist[0,k,1:],label=k)
# plt.title("distance distribution at different sequence seperations")
# plt.xlabel("distance")
# plt.ylabel("p(distance|seq_sep)")
# plt.legend()
# plt.show()

In [ ]:
#@title step 2: initialize the design model
#@markdown **Note**: we trained 5 independent models, during design we backprop through all 5 models
#@markdown for thie demo (to save time) we use (n_models=1), for best results use (n_models=5)

%%time

model = mk_design_model(add_bkg=True, n_models=1)

In [ ]:
#@title step 3: design a new sequence 
#@markdown In other words, optimize sequence to match target features derived from the PDB file

design = model.design(inputs={"bkg":_bkg[None]}, sample=True, opt_iter=200, return_traj=True)

In [29]:
#@title step 4: analyze
#@markdown the reported loss it the KL (Kullback–Leibler divergence) between
#@markdown the predicted-features and the background-features

#@markdown `loss = -pred * log(pred/bkg)`

#@markdown  `pred*log(bkg)` : maximize difference between pred and bkg
#@markdown `-pred*log(pred)`: minimize the entropy (push peaks to be sharp)
#
-0.25 * (design["feat"] * np.log(design["feat"]/_bkg)).sum(-1).mean()

-1.4350003004074097

In [ ]:
#@markdown Let's look at the predicted features of the designed protein

plot_feat(design["feat"])

amino acid sequence of the designed protein

In [ ]:
#@markdown Let's take a look at the amino acid sequence.
#@markdown You can predict its structure using AlphaFold
#@markdown if you're curious.

N_to_AA(design["I"][0].argmax(-1))

In [ ]:
#@title Animation time! 
#@markdown Here we monitor how the structure (represented as a distance matrix), 
#@markdown and sequence (represented as one-hot vector) 
#@markdown changes during optimzation. 
#@markdown For fun we also show the gradient, 
#@markdown that tells the optimizer how to change the sequence to match the desired loss function.

HTML(animate(design))

In [ ]:
#@title Visualize your result

get_pdb(design,"my_design.pdb")